<a href="https://colab.research.google.com/github/vinnik-dmitry07/Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!nvidia-smi

Sat Feb 13 17:27:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!pip install --quiet parlai

from datetime import timedelta
from pathlib import Path

GDRIVE_ROOT = Path('/content/drive/MyDrive/')

In [15]:
# Change values only in this cell!
SAVE_DIR = GDRIVE_ROOT / 'chatbot_model'  # change chatbot_model to whatever you want
EPISODE_DT = timedelta(minutes=3)  # change to split messages in separate dialogues if time delta is greater than

In [1]:
from google.colab import drive

drive.mount(GDRIVE_ROOT.parent)

Mounted at /content/drive


In [9]:
import json

with open(GDRIVE_ROOT/'result.json', 'r', encoding='utf8') as f:
    raw_messages = json.load(f)['messages']

In [10]:
from datetime import datetime

filtered_messages = []
for msg in raw_messages:
    if (
            'from' in msg and
            'from_id' in msg and
            'mime_type' not in msg and
            msg['text'] and
            isinstance(msg['text'], str)
    ):
        msg['date'] = datetime.strptime(msg['date'], '%Y-%m-%dT%H:%M:%S')
        filtered_messages.append(msg)

In [11]:
joined_messages = [filtered_messages[0]]
for i in range(1, len(filtered_messages)):
    if (
            filtered_messages[i - 1]['from_id'] == filtered_messages[i]['from_id'] and
            filtered_messages[i - 1]['date'] - filtered_messages[i]['date'] <= EPISODE_DT
    ):
        joined_messages[-1]['text'] += ' ' + filtered_messages[i]['text']
    else:
        joined_messages.append(filtered_messages[i])

In [12]:
def partition(alist, indices):
    return [alist[a:b] for a, b in zip([0] + indices, indices + [None])]

In [13]:
def save_jsonl(messages, suffix, human_readable=False):
    time_diffs = [messages[i + 1]['date'] - messages[i]['date'] for i in range(len(messages) - 1)]
    split_positions = [i + 1 for i in range(len(time_diffs)) if time_diffs[i] > EPISODE_DT]
    episodes = partition(messages, split_positions)
    print(f'{suffix} episodes: {len(episodes)}, messages: {len(messages)}')

    with open(f'data_{suffix}.jsonl', 'w', **({'encoding': 'utf8'} if human_readable else {})) as outfile:
        for episode in episodes:
            dialog = [
                {
                    'id': i % 2,
                    'text': msg['text'].replace('\n', ' '),
                } for i, msg in enumerate(episode)
            ]

            episode = {'dialog': [dialog]}
            json.dump(episode, outfile, **({'ensure_ascii': False} if human_readable else {}))
            outfile.write('\n')

In [14]:
import numpy as np

train_part, test_part, valid_part = 0.996, 0.002, 0.002
assert train_part + test_part + valid_part == 1

train, test, valid = np.split(joined_messages, [
    int(train_part * len(joined_messages)),
    int((train_part + test_part) * len(joined_messages)),
])

save_jsonl(train, suffix='train')
save_jsonl(test, suffix='test')
save_jsonl(valid, suffix='valid')


train episodes: 346, messages: 861230
test episodes: 1, messages: 1729
valid episodes: 1, messages: 1730


In [15]:
import os
os.environ['SAVE_DIR'] = str(SAVE_DIR)
!rm --recursive --force $SAVE_DIR
!mkdir --parents $SAVE_DIR

from parlai.scripts.train_model import TrainModel

TrainModel.main(
    task='jsonfile',
    jsonfile_datapath='data',
    jsonfile_datatype_extension=True,

    model='transformer/generator',
    model_file='from_pretrained/model',
    
    init_model='zoo:tutorial_transformer_generator/model',

    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file='zoo:tutorial_transformer_generator/model.dict',
    learn_positional_embeddings=True,
    
    lr=1e-5, optimizer='adam',
    warmup_updates=600,
    validation_metric='ppl',
    validation_every_n_secs=60 * 60,
    save_every_n_secs=10 * 60,

    batchsize=12, fp16=True, fp16_impl='mem_efficient',
    
    skip_generation=True,
    
    dynamic_batching='full',

    # label_turns='both',
)

17:27:55 | building data: /usr/local/lib/python3.6/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz
17:27:55 | Downloading http://parl.ai/downloads/_models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz to /usr/local/lib/python3.6/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz


/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


17:28:29 | building dictionary first...
17:28:29 | No model with opt yet at: from_pretrained/model(.opt)
17:28:29 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,datapath: /usr/local/lib/python3.6/dist-packages/data,tensorboard_logdir: None,jsonfile_datapath: data,jsonfile_datatype_extension: True,label_turns: both,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,beam_block_full_context: True,beam_length_penalty: 0.65,topk: 10,topp: 0.9,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,compute_tokenized_bleu: False,interactive_mode: False,fp16_impl: mem_efficient,force_fp16_tokens: False,adafactor_eps: (1e-30, 0.001),history_reversed: False,history_add_global_end_token: None,special_tok_lst: None,bpe_vocab: None,bpe_merge: None,bpe_add_prefix_space: None,hf_skip_special_tokens: True,max_lr_steps: -1,invsqrt_lr_

/usr/local/lib/python3.6/dist-packages/parlai/utils/fp16.py:487: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


17:28:53 | Overflow: setting loss scale to 32768.0
17:28:58 | Overflow: setting loss scale to 16384.0
17:29:02 | time:10s total_exs:5252 epochs:0.01
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
   .9091  3716 12125 518.2 5252             26314  10.97    .5458  4.12 5.509e-07 875.6  2857 61.56      .3009   
    total_train_updates  tpb   tps   ups  
                     33 4591 14982 3.263

17:29:12 | time:21s total_exs:8448 epochs:0.01
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
       1  5121 17786 308.4 3196             16384   12.3    .5493 3.968 1.151e-06 535.9  1861 52.87      .3163   
    total_train_updates  tpb   tps   ups  
                     69 5656 19648 3.474

17:29:22 | time:31s total_exs:10828 epochs:0.01
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
       1  5841 2

KeyboardInterrupt: ignored